#Исходный код RAG'a

In [1]:
#Сам добовлял устанавливаемые библиотеки для рассчета метрик
!pip install gigachain -q
!pip install sentence-transformers -q
!pip install faiss-cpu -q
!pip install gigachain_community -q
!pip install gigachain-core -q
!pip install pypdf -q
!pip install nltk -q
!pip install rouge-score -q
!pip install transformers datasets -q
!pip install torchmetrics -q
!pip install evaluate -q
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 278.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.1/341.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2

In [2]:
from getpass import getpass
from langchain.chat_models import GigaChat

In [ ]:
gigachat_api_key = getpass(prompt='Введите API ключ от GigaChat')

In [4]:
# Инициализируем языковую модель GigaChat
# verify_ssl_certs=False – без использования сертификатов Минцифры
llm = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01)

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

In [6]:
# Список путей к документам с ответами на вопросы
document_paths_new = [
    "8).pdf",
    "18).pdf",
    "24).pdf",
    "25).pdf",
    "28).pdf"
]

In [7]:
# Загрузка всех документов
from langchain_community.document_loaders import PyPDFLoader
documents = []
for path in document_paths_new:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())

In [ ]:
# Разделение текста на части
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_docs = text_splitter.split_documents(documents)
print(f"Total documents: {len(split_docs)}")

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru"
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_documents(split_docs, hf_embeddings_model)

##Создание списков вопросов и ответов данных и сгенерированных LLM

In [11]:
from langchain.chains import RetrievalQA

In [12]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [13]:
import pandas as pd

In [14]:
questions_df = pd.read_excel("q_data.xlsx", nrows=3).Question
answers_df = pd.read_excel("q_data.xlsx", nrows=3).Answer

In [15]:
answers = []
for e in answers_df:
  answers.append(e)

questions = []
for e in questions_df:
  questions.append(e)

In [ ]:
message = []
for e in questions:
    message.append(qa_chain({"query": e})['result'])

#BLEUMetric

Вторая попытка реализации (Удачная)

In [ ]:
#Попытался написать функцию, что то не работает, скорее всего из-за ссылок, но тратить время не решение проблемы не стал
#def qwer(text):
#  res = []
#  for e in range(len(text)):
#    res.append(text[e].split)
#  return res

In [ ]:
#Импортируем функцию вычисления BLEU
from nltk.translate.bleu_score import corpus_bleu

#Подготовка данных в необходимую форму для вычесления метрики BLEU (приведение к токенизированному виду)
translations = []
for e in range(len(answers)):
  translations.append(answers[e].split())

#Оборачиваем ответы в необходимую форму для вычесления метрики BLEU
references = [[[ref]] for ref in message]

#Костыльное решение, такое, чтобы привелось в необходумую форму
references_list = []
i = 0
for e in range(len(references)):
  references_list.append([])
  references_list[i].append(references[e][0][0].split())
  i = i + 1

#Вычисляем BLEU Score
bleu_score_corpus = corpus_bleu(references_list, translations)
print("Corpus BLEU Score: ", bleu_score_corpus)

In [ ]:
#Откуда взят код
references = [['I', 'love', 'eating', 'ice', 'cream'], ['He', 'enjoys', 'eating', 'cake']]
translations = [['I', 'love', 'eating', 'ice', 'cream'], ['He', 'likes', 'to', 'eat', 'cake']]

# Create a list of reference lists
references_list = [[ref] for ref in references]

#Вычисляем BLEU Score
bleu_score_corpus = corpus_bleu(references_list, translations)
print("Corpus BLEU Score: ", bleu_score_corpus)

#ROUGE

rougeL, другой компонент оценки ROUGE, вычисляет самую длинную общую подпоследовательность (LCS) между системными и справочными сводками. В отличие от N-грамм, LCS измеряет максимальную последовательность слов (не обязательно смежных), которые появляются в обоих сводках. Он предлагает более гибкую меру сходства и помогает фиксировать общую информацию, выходящую за рамки строгих дословных совпадений.

ROUGE-N - это компонент оценки ROUGE, который количественно определяет перекрытие N-граммов, непрерывных последовательностей из N элементов (обычно слов или символов), между сгенерированной системой сводкой и справочной сводкой. Он дает представление о точности и повторяемости выходных данных системы, учитывая совпадающие N-граммовые последовательности. Разделяется на rouge1, который ищёт совпадения только для отдельных слов и rouge2 - ~ для двух последовательно идущих слов.

ROUGE-S фокусируется на пропуск-биграммах. Пропуск-биграмма - это пара слов в предложении, которая допускает пробелы или слова между ними. Этот компонент определяет перекрытие пропусков и биграмм между системным и справочным резюме, позволяя оценить сходство структуры на уровне предложений. Он может фиксировать перефразирующие связи между предложениями и давать представление о способности системы передавать информацию с гибким порядком слов.

In [20]:
from rouge_score import rouge_scorer

In [21]:
r_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [22]:
df = pd.DataFrame(columns=['Question', 'Answer', 'LLM_Answer', 'rouge1', 'rouge2', 'rougeL'])
for e in range(len(questions)):
  candidate_summary = message[e]
  reference_summary = answers[e]
  r_scores = r_scorer.score(reference_summary, candidate_summary)
  new_row = {'Question': questions[e], 'Answer': answers[e], 'LLM_Answer': message[e], 'rouge1': r_scores['rouge1'], 'rouge2': r_scores['rouge2'], 'rougeL': r_scores['rougeL']}
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
df

In [ ]:
r_scores

In [ ]:
#Код, на который опирался
candidate_summary = "the cat was found under the bed"
reference_summary = "the cat was under the bed"
r_scores = r_scorer.score(reference_summary, candidate_summary)
for key in r_scores:
    print(f'{key}: {r_scores[key]}')
r_scores['rouge1']

#Семантическая похожесть на основе Sentence Transformers

##Опора

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Example sentences (already preprocessed)
tokens1 = ["[CLS]", "i", "like", "coding", "in", "python", ".", "[SEP]"]
tokens2 = ["[CLS]", "python", "is", "my", "favorite", "programming", "language", ".", "[SEP]"]


In [ ]:
# Convert tokens to input IDs
input_ids1 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens1)).unsqueeze(0)  # Batch size 1
input_ids2 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens2)).unsqueeze(0)  # Batch size 1

In [ ]:
# Obtain the BERT embeddings
with torch.no_grad():
    t_outputs1 = model(t_input_ids1)
    t_outputs2 = model(t_input_ids2)
    t_embeddings1 = t_outputs1.last_hidden_state[:, 0, :]  # [CLS] token
    t_embeddings2 = t_outputs2.last_hidden_state[:, 0, :]  # [CLS] token


In [ ]:
# Calculate similarity
t_similarity_score = cosine_similarity(t_embeddings1, t_embeddings2)
print("Similarity Score:", t_similarity_score)

##Реализация

In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
model = AutoModel.from_pretrained("deepvk/USER-bge-m3")

In [27]:
# Example sentences (already preprocessed)
tokens_answer = []
for e in range(len(answers)):
  tokens_answer.append(answers[e].split())

tokens_message = []
for e in range(len(message)):
  tokens_message.append(message[e].split())

In [28]:
# Convert tokens to input IDs
input_ids_messages = []
input_ids_answers = []

for e in range(len(tokens_answer)):
  input_ids_messages.append(torch.tensor(tokenizer.convert_tokens_to_ids(tokens_message[e])).unsqueeze(0))  # Batch size 1
  input_ids_answers.append(torch.tensor(tokenizer.convert_tokens_to_ids(tokens_answer[e])).unsqueeze(0))  # Batch size 1

In [29]:
# Obtain the BERT embeddings
outputs_messages = []
outputs_answers = []
embeddings_messages = []
embeddings_answers = []
with torch.no_grad():
    for e in range(len(tokens_answer)):
      outputs_messages.append(model(input_ids_messages[e]))
      outputs_answers.append(model(input_ids_answers[e]))
      embeddings_messages.append(outputs_messages[e].last_hidden_state[:, 0, :])  # [CLS] token
      embeddings_answers.append(outputs_answers[e].last_hidden_state[:, 0, :])  # [CLS] token

In [ ]:
# Calculate similarity
similarity_score = []
i = 0
average_similarity_score = 0
df_sim_score = pd.DataFrame(columns=['Answer', 'LLM_Answer', 'Similarity Score'])
for e in range(len(tokens_answer)):
  similarity_score.append(cosine_similarity(embeddings_messages[e], embeddings_answers[e]))
  average_similarity_score += similarity_score[e]
  i +=1
  new_row = {'Answer': answers[e], 'LLM_Answer': message[e], 'Similarity Score': similarity_score[e]}
  df_sim_score = pd.concat([df_sim_score, pd.DataFrame([new_row])], ignore_index=True)
print("Average Similarity Score = ", average_similarity_score / i)
df_sim_score

#BERTScore

In [ ]:
from torchmetrics.functional.text.bert import bert_score
preds = message
target = answers
df_BERTScore = pd.DataFrame(columns=['Answer', 'LLM_Answer', 'Precision', 'Recall', 'F1'])
b_score = bert_score(preds=preds , target=target, model_name_or_path="deepvk/USER-bge-m3")
for e in range(len(message)):
  new_row = {'Answer': answers[e], 'LLM_Answer': message[e], 'Precision': b_score['precision'][e], 'Recall': b_score['recall'][e], 'F1': b_score['f1'][e]}
  df_BERTScore = pd.concat([df_BERTScore, pd.DataFrame([new_row])], ignore_index=True)
df_BERTScore

In [ ]:
#На что опирался
from torchmetrics.functional.text.bert import bert_score
preds = ["hello there", "general kenobi", "я был здесь"]
target = ["hello there", "master kenobi", "я был тут"]
q = bert_score(preds, target)
q

#METEOR

In [29]:
import evaluate

In [ ]:
meteor = evaluate.load("meteor")

In [ ]:
meteor_res = []
meteor_res_avg = 0
i = 0
df_meteor = pd.DataFrame(columns=['Answer', 'LLM_Answer', 'meteor_results'])

predictions = [[pred] for pred in message]
references = [[ref] for ref in answers]

for e in range(len(answers)):
  meteor_res.append(meteor.compute(predictions=predictions[e], references=references[e]))
  meteor_res_avg += meteor_res[e]['meteor']
  new_row = {'Answer': answers[e], 'LLM_Answer': message[e], 'meteor_results': meteor_res[e]}
  df_meteor = pd.concat([df_meteor, pd.DataFrame([new_row])], ignore_index=True)
  i = i + 1

meteor_res_avg = meteor_res_avg / i
print(meteor_res_avg)
df_meteor

#TER

In [93]:
import sacrebleu

In [94]:
ter = evaluate.load("ter")

In [ ]:
ref = [[ref] for ref in answers]
results = ter.compute(predictions=message,
                        references=ref,
                        case_sensitive=True)
print(results)

#chrF, chrF++

In [ ]:
import evaluate

In [ ]:
ref = [[ref] for ref in answers]
chrf = evaluate.load("chrf")
results = chrf.compute(predictions=message, references=ref)
print(results)